# Test pricing transition on localnet

This notebook connects to a running localnet node, inspects pricing transition behavior,
and compares current vs scheduled prices around transition end.

## Configure connection

In [ ]:
CookieEnv = os:getenv("LOCALNET_COOKIE"),
Cookie =
    case CookieEnv of
        false ->
            'localnet';
        Value ->
            list_to_atom(Value)
    end,
NodeNameEnv = os:getenv("LOCALNET_NODE_NAME"),
NodeName0 =
    case NodeNameEnv of
        false ->
            "main-localnet";
        Value ->
            Value
    end,
{NamePart, HostPart} =
    case string:split(NodeName0, "@") of
        [Name, Host] ->
            {Name, Host};
        _ ->
            Host =
                case inet:gethostname() of
                    {ok, Name} ->
                        Name;
                    _ ->
                        "localhost"
                end,
            {NodeName0, Host}
    end,
Node = list_to_atom(NamePart ++ "@" ++ HostPart),

erlang:set_cookie(node(), Cookie).

## Local modules

The Erlang kernel compiles the project and adds local build paths automatically.

In [ ]:
ar_node:get_height().

## Compute transition window

In [ ]:
TransitionStart = rpc:call(Node, ar_pricing_transition, transition_start_2_7_2, []),
TransitionLength = rpc:call(Node, ar_pricing_transition, transition_length_2_7_2, []),
TransitionEnd = TransitionStart + TransitionLength,
StartHeight = TransitionEnd - 100,
StopHeight = TransitionEnd + 100,

#{
    transition_start => TransitionStart,
    transition_end => TransitionEnd,
    start_height => StartHeight,
    stop_height => StopHeight
}.

## Mine blocks around the transition end

In [ ]:
CurrentHeight = rpc:call(Node, ar_node, get_height, []),
case CurrentHeight < StopHeight of
    true ->
        ok = rpc:call(Node, ar_localnet, mine_until_height, [StopHeight]),
        rpc:call(Node, ar_node, get_height, []);
    false ->
        CurrentHeight
end.

## Fetch prices around transition end

In [ ]:
GetBlock =
    fun(Height) ->
        BlockHash = rpc:call(Node, ar_block_index, get_element_by_height, [Height]),
        case rpc:call(Node, ar_block_cache, get, [block_cache, BlockHash]) of
            not_found ->
                {error, {block_not_found, Height}};
            Block ->
                Block
        end
    end,

ToInt =
    fun(Value) ->
        case Value of
            Bin when is_binary(Bin) ->
                binary_to_integer(Bin);
            Int when is_integer(Int) ->
                Int;
            _ ->
                undefined
        end
    end,

GetFields =
    fun(Block) ->
        {Fields} = rpc:call(Node, ar_serialize, block_to_json_struct, [Block]),
        Fields
    end,

GetPriceInfo =
    fun(Height) ->
        case GetBlock(Height) of
            {error, _} = Error ->
                #{height => Height, error => Error};
            Block ->
                Fields = GetFields(Block),
                Price = ToInt(proplists:get_value(price_per_gib_minute, Fields)),
                Scheduled = ToInt(proplists:get_value(scheduled_price_per_gib_minute, Fields)),
                Denomination = ToInt(proplists:get_value(denomination, Fields)),
                #{height => Height, price => Price, scheduled_price => Scheduled,
                    denomination => Denomination}
        end
    end,

Heights = lists:seq(StartHeight, StopHeight),
Prices = [GetPriceInfo(Height) || Height <- Heights],
Prices.

## Validate price fields via recalc

In [ ]:
ValidateHeight =
    fun(Height) ->
        case {GetBlock(Height - 1), GetBlock(Height)} of
            {{error, _} = Error, _} ->
                {Height, Error};
            {_, {error, _} = Error} ->
                {Height, Error};
            {PrevBlock, Block} ->
                PrevFields = GetFields(PrevBlock),
                CurrFields = GetFields(Block),
                PrevDenomination = ToInt(proplists:get_value(denomination, PrevFields)),
                Denomination = ToInt(proplists:get_value(denomination, CurrFields)),
                {ExpectedPrice, ExpectedScheduled} =
                    rpc:call(Node, ar_pricing, recalculate_price_per_gib_minute, [PrevBlock]),
                ExpectedPrice2 =
                    rpc:call(Node, ar_pricing, redenominate,
                        [ExpectedPrice, PrevDenomination, Denomination]),
                ExpectedScheduled2 =
                    rpc:call(Node, ar_pricing, redenominate,
                        [ExpectedScheduled, PrevDenomination, Denomination]),
                Price = ToInt(proplists:get_value(price_per_gib_minute, CurrFields)),
                Scheduled = ToInt(proplists:get_value(scheduled_price_per_gib_minute, CurrFields)),
                case {ExpectedPrice2 == Price, ExpectedScheduled2 == Scheduled} of
                    {true, true} ->
                        ok;
                    _ ->
                        {Height, {price_mismatch, #{expected => ExpectedPrice2,
                            actual => Price, expected_scheduled => ExpectedScheduled2,
                            actual_scheduled => Scheduled}}}
                end
        end
    end,

Checks = [ValidateHeight(Height) || Height <- lists:seq(StartHeight + 1, StopHeight)],
Failures = [Check || Check <- Checks, Check =/= ok],
case Failures of
    [] ->
        ok;
    _ ->
        Failures
end.

## Validate v2 pricing post-transition

In [ ]:
ValidateV2 =
    fun(Height) ->
        case rpc:call(Node, ar_pricing_transition, is_v2_pricing_height, [Height]) of
            true ->
                case GetBlock(Height) of
                    {error, _} = Error ->
                        {Height, Error};
                    Block ->
                        Fields = GetFields(Block),
                        Price = ToInt(proplists:get_value(price_per_gib_minute, Fields)),
                        V2Price = rpc:call(Node, ar_pricing, get_v2_price_per_gib_minute,
                            [Height, Block]),
                        case Price == V2Price of
                            true ->
                                ok;
                            false ->
                                {Height, {v2_price_mismatch, #{expected => V2Price, actual => Price}}}
                        end
                end;
            false ->
                ok
        end
    end,

V2Checks = [ValidateV2(Height) || Height <- lists:seq(StartHeight, StopHeight)],
V2Failures = [Check || Check <- V2Checks, Check =/= ok],
case V2Failures of
    [] ->
        ok;
    _ ->
        V2Failures
end.

## Plot prices